# 3A.mr - Reservoir Sampling distribué - énoncé

[Reservoir Sampling](https://en.wikipedia.org/wiki/Reservoir_sampling) sur Map/Reduce. Cet algorithme est un algorithme d'échantillonnage en streaming.

In [1]:
from jyquickhelper import add_notebook_menu
add_notebook_menu()

## sampling sur Hadoop / PIG

Voir [SAMPLE](https://pig.apache.org/docs/r0.7.0/piglatin_ref2.html#SAMPLE). En pratique, il difficile d'écrire un algorithme plus efficace à partir d'un langage haut niveau si ce langage haut niveau implémente lui-même la fonctionnalité. C'est une façon de reconnaître que c'est un besoin fréquent et qu'il est recommandé d'utiliser la fonction native.

## Reservoir Sampling

Le [reservoir sampling](https://en.wikipedia.org/wiki/Reservoir_sampling) est une façon de tirer un échantillon aléatoire de $k$ nombres parmi $N$ nombres qui ne nécessite pas de conserver ces $N$ nombres en mémoire.

In [3]:
ensemble = [ "%d%s" % (i, chr(i%26 + 97)) for i in range(0,10000)]
ensemble[:5]

['0a', '1b', '2c', '3d', '4e']

In [4]:
import random
def reservoir_sampling(ensemble, k):
    N = len(ensemble)
    echantillon = []
    for i, e in enumerate(ensemble):
        if len(echantillon) < k:
            echantillon.append(e)
        else:
            j = random.randint(0, i)
            if j < k:
                echantillon[j] = e
    return echantillon

reservoir_sampling(ensemble, 10)

['9128c',
 '4112e',
 '7196u',
 '7941l',
 '6722o',
 '7465d',
 '5715v',
 '1128k',
 '3054m',
 '8943z']

Le *reservoir sampling* reproduit un tirage sans remise. L'algorithme habituel - on tire $k$ éléments parmi $N$, la probabilité qu'un élément fasse partie de l'échantillon est de :

$$p = \frac{C_{N-1}^{k-1}}{C_N^k} = \frac{ \frac{N-1!}{k-1! N-k!} } { \frac{N!}{k! N-k!} } = \frac{ N-1! k! } { N! k-1!} = \frac{k}{N}$$

Dans le cas du *reservoir sampling*, on doit s'assurer que chaque élément à la même probabilité de faire partie de l'échantillon. On procède par récurrence. Le résultat est vrai pour $k=N$. On suppose que est vrai à l'ordre $N-1$. La probabilité qu'un élément de l'ensemble fasse partie de l'échantillon est $\frac{k}{N-1}$. A l'ordre $N$, il y a une probabilité $\frac{k}{N}$ de remplacer un des éléments de l'échantillon. L'élément $N$ a donc une probabilité $\frac{k}{N}$ de faire partie de l'échantillon. Pour les éléments faisant déjà partie de l'échantillon, leur probabilité de rester est $\frac{N-k}{N} + \frac{k}{N}\frac{k-1}{k}=\frac{N-1}{N}$. La probabilité qu'un élément présent dans l'échantillon soit dans l'échantillon à l'itération $N$ est de $\frac{N-1}{N} \frac{k}{N-1}=\frac{k}{N}$. L'échantillon produit par le *reservoir sampling* a les mêmes propriétés qu'un échantillon produit avec un tirage sans remise.

Cet algorithme est intéressent lorsqu'on veut échantillonner sur une grande base de données car il nécessite de ne garder en mémoire que l'échantillon.

## Reservoir Sampling Distribué

Distribuer l'algorithme paraît simple : une partie des données ira sur une machine qui en tirera un échantillon, l'autre machine tirera un échantillon aléatoire sur l'autre partie des données. Par extension, sur $m$ machines, on obtient $m$ échantillons de tailles $k_1, ..., k_m$ tirés sur des ensembles de tailles $N_1, ..., N_m$. Il faut s'assurer que la version distribuée produit un échantillon avec les mêmes propriétés.

### exercice 1 : combinaison

Comment recombiner ces échantillons aléatoires ? Comment choisir les $k_i$ sachant qu'on doit tirer un échantillon de taille $k \leqslant \sum_{i=1}^m k_i$ parmi $N=\sum_{i=1}^m N_i$ ?

### exercice 2 : script PIG

Ecrire le script PIG correspondant à l'algorithme.

## Petit problème théorique

Un [Générateur congruentiel linéaire](https://fr.wikipedia.org/wiki/G%C3%A9n%C3%A9rateur_congruentiel_lin%C3%A9aire) imite seulement le hasard, ce n'est pas vraiment le hasard. La suite dépend d'une graine ou *seed*. Si deux machines partent de la même graine, elle produiront la même suite. Qu'en est-il du hasard ?

La solution de l'exercice précédent est-elle correcte d'un point de statistique ?